<a href="https://colab.research.google.com/github/ParkSlingshot/-/blob/main/trafficCustomYoloDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preperation

In [ ]:
!wget https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip

--2021-09-06 10:16:27--  https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1661866983 (1.5G) [application/zip]
Saving to: ‘FullIJCNN2013.zip’

FullIJCNN2013.zip   100%[===================>]   1.55G  24.6MB/s    in 65s     

2021-09-06 10:17:33 (24.3 MB/s) - ‘FullIJCNN2013.zip’ saved [1661866983/1661866983]



In [ ]:
!unzip FullIJCNN2013.zip

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2 as cv
from google.colab.patches import cv_imshow

In [ ]:
!ls -al ./FullIJCNN2013/gt.txt

-rw-r--r-- 1 root root 35282 Jul  9  2013 ./FullIJCNN2013/gt.txt


In [ ]:
test_per = 0.2
file_name = './FullIJCNN2013/gt.txt'

In [ ]:
annotation = pd.read_csv(file_name, 
                         names = ['ImageId', 'Roix1', 'Roiy1', 'Roix2', 'Roiy2', 'ClassId' ], sep=';')
annotation.head()

,ImageId,Roix1,Roiy1,Roix2,Roiy2,ClassId
0,00000.ppm,774,411,815,446,11
1,00001.ppm,983,388,1024,432,40
2,00001.ppm,386,494,442,552,38
3,00001.ppm,973,335,1031,390,13
4,00002.ppm,892,476,1006,592,39


In [ ]:
Prohibitory = [0,1,2,3,4,5,7,8,9,10,15,16]
Danger = [11,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
Mandatory = [33,34,35,36,37,38,39,40]
Other = [6,12,13,14,17,32,41,42]

categories= ['Prohibitory', 'Danger', 'Mandatory', 'Other']

In [ ]:
annotation['CategoryId'] =0

annotation.loc[annotation['ClassId'].isin(Prohibitory), 'CategoryId'] = 0
annotation.loc[annotation['ClassId'].isin(Danger), 'CategoryId'] = 1
annotation.loc[annotation['ClassId'].isin(Mandatory), 'CategoryId'] = 2
annotation.loc[annotation['ClassId'].isin(Other), 'CategoryId'] = 3

In [ ]:
annotation['CenterX'] = (annotation['Roix1']+annotation['Roix2']) /2 
annotation['CenterY'] = (annotation['Roiy1']+annotation['Roiy2']) /2 
annotation['width'] = annotation['Roix2']-annotation['Roix1']
annotation['height'] = annotation['Roiy2']-annotation['Roiy1']
annotation.head()

,ImageId,Roix1,Roiy1,Roix2,Roiy2,ClassId,CategoryId,CenterX,CenterY,width,height
0,00000.ppm,774,411,815,446,11,1,794.5,428.5,41,35
1,00001.ppm,983,388,1024,432,40,2,1003.5,410.0,41,44
2,00001.ppm,386,494,442,552,38,2,414.0,523.0,56,58
3,00001.ppm,973,335,1031,390,13,3,1002.0,362.5,58,55
4,00002.ppm,892,476,1006,592,39,2,949.0,534.0,114,116


In [ ]:
image_path = './FullIJCNN2013'

In [ ]:
os.chdir(image_path)

In [ ]:
paths = []
for current_dir, dirs, files in os.walk('.') :
  for file in files:
    if file.endswith('.ppm'):
      image_ppm = cv.imread(file)
      h, w = image_ppm.shape[:2]
      image_name = file.split('.')[0]
      txt_file = image_name+'.txt'
      image_jpg = image_name+'.jpg'
      cv.imwrite(image_jpg, image_ppm)
      txt = open(txt_file, 'w')
      objects = annotation.loc[annotation['ImageId']==file].copy()
      for index, data in objects.iterrows():
        line = str(data['CategoryId']) +' '+ str(data['CenterX']/w) +' '+str(data['CenterY']/h)+ ' '+str(data['width']/w)+ ' '+ str(data['height']/h)
        txt.write(line)
      image_path = image_jpg + '\n'
      paths.append(image_path)
      print(image_jpg, 'is created')

In [ ]:
num_test = int(len(paths)*test_per)
print(num_test)

422


In [ ]:
paths_test = paths[:num_test]
paths_train = paths[num_test:]

In [ ]:
with open('triain.txt', 'w') as train_txt:
  for path in paths_train:
    train_txt.write(path)
  print('[train.txt] is created')
  

[train.txt] is created


In [ ]:
with open('test.txt', 'w') as test_txt:
  for path in paths_test:
    test_txt.write(path)
  print('[test.txt] is created')
  

[test.txt] is created


In [ ]:
 with open('traffic.data', 'w') as data:
  data.write('classes = ' +'1\n')
  data.write('train= ' + 'train.txt' + '\n')
  data.write('validation= ' + 'test.txt' + '\n')
  data.write('names= ' +  'classes.names' + '\n')
  data.write('backup = backup')
  print('[custom_data.data] is created') 

[custom_data.data] is created


In [ ]:
!mkdir backup

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
from google.colab import files
files.upload()

Saving traffic-sign-yolo.cfg to traffic-sign-yolo.cfg


{'traffic-sign-yolo.cfg': b'[net]\n# Testing\n#batch=1\n#subdivisions=1\n# Training\nbatch=64\nsubdivisions=16\nheight=416\nwidth=416\nchannels=3\nmomentum=0.9\ndecay=0.0005\nangle=0\nsaturation = 1.5\nexposure = 1.5\nhue=.1\n\nlearning_rate=0.001\nburn_in=1000\nmax_batches = 8000\npolicy=steps\nsteps=6400,7200\nscales=.1,.1\n\n[convolutional]\nbatch_normalize=1\nfilters=32\nsize=3\nstride=1\npad=1\nactivation=leaky\n\n# Downsample\n\n[convolutional]\nbatch_normalize=1\nfilters=64\nsize=3\nstride=2\npad=1\nactivation=leaky\n\n[convolutional]\nbatch_normalize=1\nfilters=32\nsize=1\nstride=1\npad=1\nactivation=leaky\n\n[convolutional]\nbatch_normalize=1\nfilters=64\nsize=3\nstride=1\npad=1\nactivation=leaky\n\n[shortcut]\nfrom=-3\nactivation=linear\n\n# Downsample\n\n[convolutional]\nbatch_normalize=1\nfilters=128\nsize=3\nstride=2\npad=1\nactivation=leaky\n\n[convolutional]\nbatch_normalize=1\nfilters=64\nsize=1\nstride=1\npad=1\nactivation=leaky\n\n[convolutional]\nbatch_normalize=1\nf

In [ ]:
!chmod +x ./darknet

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-09-06 10:56:55--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  54.3MB/s    in 2.9s    

2021-09-06 10:56:58 (54.3 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!./darknet detector train traffic.data traffic-sign-yolo.cfg darknet53.conv.74 -dont_show

valid: Using default 'train.txt'
traffic-sign-yolo
layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32 0.299 BF
   1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64 1.595 BF
   2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32 0.177 BF
   3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1
   5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128 1.595 BF
   6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
   7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5
   9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
  10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
  11 Shortcut Layer: 8
  12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x  52 x 256 1.595 BF
  13 conv    128  1 x 1 / 1    52

In [ ]:
!ls -al

total 3345136
drwxr-xr-x 46 root root     73728 Sep  6 10:56 .
drwxr-xr-x  1 root root      4096 Sep  6 10:18 ..
drwxr-xr-x  2 root root      4096 Jul  9  2013 00
-rw-r--r--  1 root root    361627 Sep  6 10:47 00000.jpg
-rw-r--r--  1 root root   3264016 Nov 26  2012 00000.ppm
-rw-r--r--  1 root root        58 Sep  6 10:47 00000.txt
-rw-r--r--  1 root root    520885 Sep  6 10:47 00001.jpg
-rw-r--r--  1 root root   3264016 Nov 26  2012 00001.ppm
-rw-r--r--  1 root root       166 Sep  6 10:47 00001.txt
-rw-r--r--  1 root root    400637 Sep  6 10:47 00002.jpg
-rw-r--r--  1 root root   3264016 Nov 26  2012 00002.ppm
-rw-r--r--  1 root root        52 Sep  6 10:47 00002.txt
-rw-r--r--  1 root root    623796 Sep  6 10:47 00003.jpg
-rw-r--r--  1 root root   3264016 Nov 26  2012 00003.ppm
-rw-r--r--  1 root root       174 Sep  6 10:47 00003.txt
-rw-r--r--  1 root root    578250 Sep  6 10:47 00004.jpg
-rw-r--r--  1 root root   3264016 Nov 26  2012 00004.ppm
-rw-r--r--  1 root root       111 Sep  